In [18]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv(r'C:\Users\HP\jigsaw-toxic-comment-classification-challenge\train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [19]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
train['none'] = 1-train[labels].max(axis=1)
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [20]:
# To lowercase all the text 
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# To remove all punctuations and special characters
train['comment_text'] = train['comment_text'].str.replace('[^\w\s]','')
train.loc[2, 'comment_text']

'hey man im really not trying to edit war its just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info'

In [21]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
train.loc[2, 'comment_text']

'hey man im trying edit war guy constantly removing relevant information talking edits instead talk page care formatting actual info'

In [22]:
from sklearn.model_selection import train_test_split
labels = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'none']
x = train['comment_text']
y = train[labels]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 0)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 50000
embedding_dim = 32
max_length = 240

tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<oov>")
tokenizer.fit_on_texts(x_train)
print('vocabulary is:', len(tokenizer.word_index))
x_train = (tokenizer.texts_to_sequences(x_train))
x_test = (tokenizer.texts_to_sequences(x_test))
print('Train Set: ', len(x_train))
print('Test Set:' , len(x_test))
x_train = pad_sequences(x_train, padding = 'post', maxlen=max_length)
x_test = pad_sequences(x_test, padding = 'post', maxlen=max_length)

vocabulary is: 217942
Train Set:  127656
Test Set: 31915


In [24]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(7, activation = 'softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 30
history = model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), verbose = 2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 240, 32)           1600000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 240, 32)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 455       
Total params: 1,658,375
Trainable params: 1,658,375
Non-trainable params: 0
_________________________________________________________________
Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 - 5127s - loss: 0.4708 - accuracy:

In [25]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(7, activation = 'softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.summary()

num_epochs = 30
history = model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), verbose = 2)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 240, 32)           1600000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 240, 32)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 903       
Total params: 1,667,079
Trainable params: 1,667,079
Non-trainable params: 0
_________________________________________________________________
Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 - 1253s - loss: 0.8660 - accuracy:

# Test the Model

In [48]:
test_features = pd.read_csv(r'C:\Users\HP\jigsaw-toxic-comment-classification-challenge\test.csv')
test_labels = pd.read_csv(r'C:\Users\HP\jigsaw-toxic-comment-classification-challenge\test_labels.csv')

In [49]:
# Concatination of Test Features and Test Labels
test = pd.concat([test_features, test_labels], axis = 1)

# Removal of data points which were incorrectly labeled
columns = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
for column in columns:
    test = test.drop(test[test[column] == -1 ].index, axis = 0)

test.head()

,id,comment_text,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",00059ace3e3e9a53,0,0,0,0,0,0


In [57]:
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test['comment_text'] = test['comment_text'].str.replace('[^\w\s]','')
spacy_nlp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
# Adding adition category for such comments that do no belong to any categories in the data
labels = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
test['none'] = 1-train[labels].max(axis=1)
test.head()

,id,comment_text,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
5,0001ea8717f6de06,thank understanding think highly revert discus...,0001ea8717f6de06,0,0,0,0,0,0,1
7,000247e83dcc1211,dear god site horrible,000247e83dcc1211,0,0,0,0,0,0,1
11,0002f87b16116a7f,somebody invariably try add religion mean way ...,0002f87b16116a7f,0,0,0,0,0,0,1
13,0003e1cccfd5a40a,says right type type institution needed case l...,0003e1cccfd5a40a,0,0,0,0,0,0,1
14,00059ace3e3e9a53,adding new product list sure relevant adding n...,00059ace3e3e9a53,0,0,0,0,0,0,1


In [58]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'none']
x_test = test['comment_text']
y_test = test[labels]

x_test = (tokenizer.texts_to_sequences(x_test))
x_test = pad_sequences(x_test, padding = 'post', maxlen=max_length)
x_test.shape, y_test.shape

((63978, 240), (63978, 7))

In [67]:
score = model.evaluate(x_test, y_test, batch_size= 64, verbose = 2)

63978/63978 - 269s - loss: 0.9248 - accuracy: 0.8360
